## 11. Alignment of Model Scales

## Numerical experiments - Week 04/2025

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

The goal of this study is to cross-verify and combine the functionalities of the high-resolution, medium-resolution and low-resolution models, in order to obtain a universal mathematical relationship between cell wall permeability, spore density and spore coverage due to clustering.

## Prerequisite libraries